In [29]:
import os
import pandas as pd

# Step 1: Load species names from species_all.xlsx into a dictionary
species_df = pd.read_excel('h:\\Yehmh\\DNDF\\all_species_2.xlsx')
species_dict = dict(zip(species_df['Species_ch'], species_df['Species_en']))

# Step 2: Define a default value for unknown species
default_species_en = "Unknown"

# Step 3: Specify the file path containing the Excel file
file_path = 'h:\\Yehmh\\DNDF\\2023大農大富每木資料.xlsx'  # Replace with the path to your file

# Step 4: Read the file
df = pd.read_excel(file_path)

# Step 5: Map species names to English names and append a new column
df['CSP'] = df['CSP'].map(species_dict)

# Step 6: Handle unknown species
df['CSP'] = df['CSP'].fillna(default_species_en)

# Step 7: Drop the 'note' and 'code' columns
df = df.drop(columns=['Note', 'Code'])

# Step 8: Save the modified DataFrame to a new CSV file
output_file_path = 'h:\\Yehmh\\DNDF\\DNDF_tree_survey_2023.csv'  # Replace with your desired output path
df.to_csv(output_file_path, index=False)

print("Conversion complete.")


Conversion complete.


In [30]:
import pandas as pd

# Step 1: Read the coordinates file
coord_file_path = 'h:/Yehmh/DNDF/DNDF_PlantLine_twd97.csv'
coords_df = pd.read_csv(coord_file_path)

# Step 2: Read the data file
data_file_path = 'h:\\Yehmh\\DNDF\\DNDF_tree_survey_2023.csv'  # Replace with your data file path
data_df = pd.read_csv(data_file_path)

data_df['PX'] = pd.to_numeric(data_df['PX'], errors='coerce')
data_df['PY'] = pd.to_numeric(data_df['PY'], errors='coerce')
coords_df['TWD97_X'] = pd.to_numeric(coords_df['TWD97_X'], errors='coerce')
coords_df['TWD97_Y'] = pd.to_numeric(coords_df['TWD97_Y'], errors='coerce')

# Step 3: Iterate through each row in data_df and update TWD97_X and TWD97_Y
for index, data_row in data_df.iterrows():
    plot = data_row['Plot']       # Replace with the actual column name in your data
    transect = data_row['Transect']  # Replace with the actual column name in your data
    
    for _, coord_row in coords_df.iterrows():
        coord_plot = coord_row['Plot']
        coord_transect = coord_row['Transect']
        
        # Check if the current row matches the plot and transect
        if coord_plot == plot and coord_transect == transect:
            # Check if the values are not NaN
            if pd.notna(coord_row['TWD97_X']) and pd.notna(coord_row['TWD97_Y']) and pd.notna(data_row['PX']) and pd.notna(data_row['PY']):
                # Calculate new columns using the current row's values
                data_df.at[index, 'TWD97_X'] = coord_row['TWD97_X'] + data_row['PX']
                data_df.at[index, 'TWD97_Y'] = coord_row['TWD97_Y'] + data_row['PY']
            else:
                print(f"NaN values encountered for Plot: {plot}, Transect: {transect}")
            break  # No need to continue the loop once the matching row is found

# Step 4: Save the updated DataFrame to a new CSV file
output_file_path = 'h:\\Yehmh\\DNDF\\DNDF_tree_survey_2023_00.csv'  # Replace with your desired output path
data_df.to_csv(output_file_path, index=False)

print("DataFrame updated and saved.")


NaN values encountered for Plot: 65, Transect: 1
NaN values encountered for Plot: 65, Transect: 1
NaN values encountered for Plot: 65, Transect: 1
NaN values encountered for Plot: 14, Transect: 1
NaN values encountered for Plot: 146, Transect: 1
NaN values encountered for Plot: 65, Transect: 1
NaN values encountered for Plot: 65, Transect: 1
DataFrame updated and saved.


In [31]:
import math
import pandas as pd

def calculate_angle(point1, point2):
    dx = point2[0] - point1[0]
    dy = point2[1] - point1[1]
    angle_rad = math.atan2(dy, dx)
    angle_deg = math.degrees(angle_rad)
    return angle_deg

def rotate_point(point, angle_rad):
    # Perform rotation using rotation matrix
    rotated_x = point[0] * math.cos(angle_rad) - point[1] * math.sin(angle_rad)
    rotated_y = point[0] * math.sin(angle_rad) + point[1] * math.cos(angle_rad)
    return rotated_x, rotated_y

# Paths to the required files
input_csv_file = 'h:\\Yehmh\\DNDF\\DNDF_tree_survey_2023_00.csv'
plantline_coord_file = 'h:\\Yehmh\\DNDF\\DNDF_PlantLine_twd97_all.csv'
output_file = 'h:\\Yehmh\\DNDF\\DNDF_tree_survey_2023_rotated.csv'

# Load the merged CSV and plantline coordinates dataframes
combined_df = pd.read_csv(input_csv_file)
plantline_coord_df = pd.read_csv(plantline_coord_file)

print(plantline_coord_df)
print(combined_df)

# Create an empty list to store the rotated dataframes
rotated_dfs = []

# Process each unique plot and transect
for (plot, transect), group_df in combined_df.groupby(['Plot', 'Transect']):
    print(plantline_coord_df['Transect'])
    row1 = plantline_coord_df[(plantline_coord_df['Plot'] == plot) & (plantline_coord_df['Transect'] == transect) & (plantline_coord_df['block_x'] == 0) & (plantline_coord_df['block_y'] == 0)]
    row2 = plantline_coord_df[(plantline_coord_df['Plot'] == plot) & (plantline_coord_df['Transect'] == transect) & (plantline_coord_df['block_x'] == 5) & (plantline_coord_df['block_y'] == 0)]

    if row1.empty or row2.empty:
        print(f'No coordinates found for plot {plot}_{transect}')
        continue

    point1 = (row1['TWD97_X'].values[0], row1['TWD97_Y'].values[0])
    point2 = (row2['TWD97_X'].values[0], row2['TWD97_Y'].values[0])

    # Calculate angle between the line connecting the points and the horizontal line
    angle_rad = math.atan2(point2[1] - point1[1], point2[0] - point1[0])

    # Vectorized translation and rotation using pandas operations
    x = group_df['TWD97_X'] - point1[0]
    y = group_df['TWD97_Y'] - point1[1]
    group_df['TWD97_X'], group_df['TWD97_Y'] = rotate_point((x, y), angle_rad)

    # Translate back
    group_df['TWD97_X'] += point1[0]
    group_df['TWD97_Y'] += point1[1]

    # Append the modified dataframe to the list
    rotated_dfs.append(group_df)

# Concatenate all rotated dataframes into a single dataframe
final_df = pd.concat(rotated_dfs)

# Save the final dataframe to a single CSV file
final_df.to_csv(output_file, index=False)


     Plot  Transect  block_x  block_y   latitude   longitude  Unnamed: 6  \
0      12         1        5        0  23.627039  121.425218         NaN   
1      12         1        5        1  23.627222  121.425200         NaN   
2      12         1        3        1  23.627200  121.424806         NaN   
3      12         1        0        1  23.627169  121.424224         NaN   
4      12         1        0        0  23.627000  121.424242         NaN   
..    ...       ...      ...      ...        ...         ...         ...   
131   146         2        1        5  23.618529  121.407121         NaN   
132   146         2        1        3  23.618186  121.407171         NaN   
133   146         2        1        0  23.617611  121.407206         NaN   
134   146         2        0        0  23.617627  121.407006         NaN   
135   146         2        0        3  23.617993  121.407007         NaN   

         TWD97_X      TWD97_Y  
0    293385.0874  2613784.310  
1    293383.1907  26138